In [24]:
# import some common libraries
import numpy as np
import os, json, cv2, random,sys
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import ColorMode

import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/home/joon/code/gitpipeline/pipeline/model/utils')
import utils



1.10.0+cu102 True


In [16]:

# import torch
# torch.cuda.set_device(1) 

In [21]:
import pathlib
# sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(__file__))))
print(pathlib.Path().absolute())
os.path.dirname("./")

/home/joon/code/gitpipeline/pipeline/model


'.'

In [3]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
classes = ['black smoke', 'gray smoke', ' white smoke', 'fire']

In [5]:
# train_df_path = '/media/PCData/dairesearch/code/BigDataproject/forest_fire_real_time_detection/src/prepare_data/train_dfs.csv'
# train_df = pd.read_csv(train_df_path)
# # train_df.head()

In [6]:
from detectron2.data import MetadataCatalog, DatasetCatalog
DatasetCatalog.clear()

# DatasetCatalog.register("aihub_train", lambda: utils.get_dicts(train_df))
MetadataCatalog.get("aihub_train").set(thing_classes = classes)

namespace(name='aihub_train',
          thing_classes=['black smoke', 'gray smoke', ' white smoke', 'fire'])

In [7]:
aihub_metadata = MetadataCatalog.get("aihub_train")

In [23]:
from detectron2.engine import DefaultTrainer
from vovnet import add_vovnet_config

cfg = get_cfg()

In [9]:

from detectron2.utils.visualizer import ColorMode

from detectron2.utils.video_visualizer import VideoVisualizer
from tqdm import tqdm
## Access webcam

instance_mode = ColorMode.IMAGE
metadata = aihub_metadata

print(metadata)

Metadata(name='aihub_train', thing_classes=['black smoke', 'gray smoke', ' white smoke', 'fire'])


In [10]:
# dataset_dicts = utils.get_dicts(train_df)


In [11]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
add_vovnet_config(cfg)
cfg.merge_from_file("/home/joon/code/gitpipeline/pipeline/model/weights/FRCN-V2-19-slim-FPNLite-3x/config.yaml")
cfg.DATASETS.TRAIN = ("aihub_train")
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 36
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("/media/daitran/Data_SSD/code/SmartInsideAI/github/concrete_crack_detection/src/detectron/configs/faster_rcnn_V_19_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 10000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes) 
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

# os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg) 
# trainer.resume_or_load(resume=False)
# trainer.train()

cfg.MODEL.WEIGHTS = os.path.join("/home/joon/code/gitpipeline/pipeline/model/weights/FRCN-V2-19-slim-FPNLite-3x/model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
cfg.DATASETS.TEST = ("aihub_train", )
predictor = DefaultPredictor(cfg)

In [12]:
# import time
# for d in random.sample(dataset_dicts, 5):  
#     # set time
    
#     im = cv2.imread(d["file_name"])
#     print(d['file_name'])
#     # start_time = time.clock()   
#     outputs = predictor(im)
#     # print("=========INFERENCE TIME PER IMAGE=========")
#     # print(time.clock() - start_time, "seconds")

#     v = Visualizer(im[:, :, ::-1],
#                    metadata=aihub_metadata, 
#                    scale=0.8,
#                    instance_mode=ColorMode.IMAGE   
#     )
#     v = v.draw_instance_predictions(outputs["instances"].to("cpu")) #Passing the predictions to CPU from the GPU


#     plt.figure(figsize=(10,5))
#     plt.imshow(v.get_image())
#     plt.show()
#     # break
#     # plt.pause(2)
#     # plt.close()

In [14]:
cam = cv2.VideoCapture(2)

instance_mode = ColorMode.IMAGE
metadata = aihub_metadata

print(metadata)
video_visualizer = VideoVisualizer(metadata, instance_mode)

while True:
    ret, frame = cam.read()
    
    # dim = (512, 512)
    
    frame = cv2.resize(frame, None, fx=2, fy=2, interpolation=cv2.INTER_AREA)

    im = frame
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=aihub_metadata, 
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE   
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu")) #Passing the predictions to CPU from the GPU
    vis_frame = cv2.cvtColor(v.get_image(), cv2.COLOR_RGB2BGR)
    cv2.imshow('Input', vis_frame)
    # cv2.imshow('Input', frame)
    
    if cv2.waitKey(1)& 0xFF ==ord('q'):
        break
cam.release()
cv2.destroyAllWindows()

Metadata(name='aihub_train', thing_classes=['black smoke', 'gray smoke', ' white smoke', 'fire'])
